In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import docx2txt
import re
import joblib
import glob
import os
import sys
import pandas as pd
import fitz
import pypandoc
from pptx import Presentation
import glob
import docx2txt
import fitz
import re
import win32com.client as win32
from win32com.client import constants

In [2]:

def data_cleansing(content):
    # content =   docx_to_txt(filepath)
    # # documents = []

    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))

    stemmer = WordNetLemmatizer()
    

    # for sen in range(0, len(content)):

    #Remove \n from text

    document = re.sub(r'\\n', ' ', str(content))
    # print(content)
    # print(document)

    # remove \t from text
    document = re.sub(r'\\t', ' ', document)

    # Remove \\ from the text
    document = re.sub(r'\\', ' ', document)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)


    #Remove special character
    document = re.sub(r"[^a-zA-Z0-9]+", ' ', document)

    # print(document)


    # Remove one char joins with 2 digit number
    document= re.sub(r'[A-Za-z]{1}[0-9]{2}', ' ', document)
    #Remove 1 digit number joins with 2 char
    document = re.sub(r'[0-9]{1}[A-Za-z]{2}',' ',document)
    #Remove one char in between two numbers
    document = re.sub(r'[0-9]{1}[A-Za-z]{1}[0-9]{1}',' ',document)
    #Remove one number in between two char
    document = re.sub(r'[A-Za-z]{1}[0-9]{1}[A-Za-z]{1}',' ',document)

    # Remove two char joins with 2 digit number
    document= re.sub(r'[A-Za-z]{2}[0-9]{2}', ' ', document)

    # Remove two char joins with 1 digit number
    document= re.sub(r'[A-Za-z]{2}[0-9]{1}', ' ', document)
    # print(document)


    # Remove double digit number
    document=re.sub('(\\b[0-9][0-9] \\b|\\b [0-9][0-9]\\b)', ' ', document).strip()
    #Remove single character
    document=re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', ' ', document).strip()
    # Remove single digit number
    document=re.sub('(\\b[0-9] \\b|\\b [0-9]\\b)', ' ', document).strip()
    # print(document)

    # Substituting multiple spaces with single space
    document = re.sub(r' +', ' ', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    #     document = ' '.join(e for e in document if e.isalnum())
    document = ' '.join([word for word in document.split(' ') if word not in stop_words])
    document = ' '.join([w for w in document.split() if len(w)>1])
    return document
    
#     documents.append(document)

In [3]:
def pdf_txt(path):
    try:
        page=""
        doc = fitz.open(path)
        n=doc.pageCount
    #     print ("number of pages: %i")
        #print(doc.metadata)
        wrd = ['Scanned by CamScanner\n','Generated by CamScanner from intsig.com\n',
               ' \n','Scanned with CamScanner\n','Generated by CamScanner\n']
        for i in range(n):

            page1 = doc.loadPage(i)
            page1text = page1.getText()
            if (page1text not in wrd) & (not page1text[0].isdigit()):
                page+=page1text
                x = 0
#                 print('success')
            else:
                x = 1
        if x == 1:
            print("Scanned pdf")
            return 1
        else:
            print('Searchable pdf')
        return page
    except ValueError:
        print('Password protected file')
        return 1
    except:
        print('unsupported file')
        return 1

In [4]:
def docx_to_txt(path):
    try:
        text = docx2txt.process(path)
        wrd = ['Scanned by CamScanner\n','Generated by CamScanner from intsig.com\n',
               ' \n','Scanned with CamScanner\n','Generated by CamScanner\n','']
        if text not in wrd:
            x=0
        else:
            x=1
        if x == 1:
            print('Scanned docx')
            return 1
        else:
            print('Searchable docx')
        return text
    except ValueError:
        print('Password protected file')
        return 1
    except:
        print('unsupported file')
        return 1
        

In [5]:
def pptx_txt(path):
    try:
        page=""
        for eachfile in glob.glob(path):

            prs = Presentation(eachfile)
    #         print(eachfile)
    #         print("----------------------")
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
    #                     print(shape.text)
                        page+=shape.text
        return page
    except ValueError:
        print('Password protected file')
        return 1
    except:
        print('unsupported file')
        return 1

In [6]:
def main_Function(filepath):
    content = ''
#     wordcnt=0
    filename, file_extension = os.path.splitext(filepath)
    print(file_extension)
    if file_extension == '.docx':
        content = docx_to_txt(filepath)
    
    elif file_extension.lower() == ".pdf":
        content=pdf_txt(filepath)
        
    elif file_extension == ".PDF":
        content=pdf_txt(filepath)
    
#     wordcnt = len(content.split())
    if content != 1:
        content = data_cleansing(content)
        count_vectorizer_saved=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\count_vectorizer_cv.sav','rb'))    
        test_data_features=count_vectorizer_saved.transform([content])
        tfidf_saved=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\term_freq_inv_df.sav','rb'))
        test_data_features_tfidf=tfidf_saved.transform(test_data_features)
        lgbm_pickled_model=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\light_gbm.sav','rb'))
        x=lgbm_pickled_model.predict(test_data_features_tfidf)
        se=pd.Series(x)
        category = {0:'3rd Parties',1:'GMC',2:'MDU_Contacts', 3:'MDU_invoice', 4:'NHSLA'}
        result=list(se.apply(lambda x: category[x]))
        score=lgbm_pickled_model.predict_proba(test_data_features_tfidf)
        cat2=result[0]
        cat1=""

        if cat2=='3rd Parties':
            cat1="Experts"
            cat2=='3rd Party Expert'
        if cat2=='GMC':
            cat1="Registration Body"
        if cat2=='MDU_Contacts':
            cat1="MDU Solicitor"
            cat2="Contacts"
        if cat2=='MDU_invoice':
            cat1="MDU Solicitor"
            cat2="Invoices"
        if cat2=='NHSLA':
            cat1="3rd Parties"        
            cat2="Trust / NHSLA"
        score=format(max(score[0]*100), '.2f')
        bss="No"
        if float(score)<=80:
            bss="Yes"
        df = pd.DataFrame({'filename':filename.split('/')[-1]+file_extension,'Prediction_cat1':cat1,'Prediction_cat2':cat2,'score':score,'bss':bss}, index=[0])
       
    else:
        df = pd.DataFrame({'filename':[filename.split('/')[-1]+file_extension], 'Type':['Unsupported File / Scanned PDF / Password Protected']})
    return df

In [7]:
lis1 = []
lis2 = []
path= r'C:/ML Training Docs/test/test/files/'
for filename in os.listdir(path):
    file_path = os.path.join(path,filename)
    x = main_Function(file_path)
    if x.shape[1] == 5:
        lis1.append(x)
    else:
        lis2.append(x)

df1 = pd.concat(lis1, axis=0)
df2 = pd.concat(lis2, axis=0)

.PDF
Searchable pdf


mupdf: cannot find startxref


.docx
unsupported file
.docx
Searchable docx
.pdf
Searchable pdf
.PDF
Searchable pdf


mupdf: cannot find startxref


.docx
Scanned docx
.pdf
Scanned pdf
.pdf
unsupported file
.docx
Searchable docx
.pdf
Scanned pdf
.docx
Searchable docx
.pdf
unsupported file
.pdf
Password protected file


In [12]:
df1

,filename,Prediction_cat1,Prediction_cat2,score,bss
0,110012.PDF,MDU Solicitor,Contacts,88.57,No
0,1710093.docx,3rd Parties,Trust / NHSLA,99.85,No
0,180014.pdf,Registration Body,GMC,72.15,Yes
0,180579.PDF,MDU Solicitor,Invoices,99.71,No
0,490001.docx,Registration Body,GMC,47.88,Yes
0,550172.docx,MDU Solicitor,Invoices,99.82,No


In [13]:
df2

,filename,Type
0,1630070.docx,Unsupported File / Scanned PDF / Password Prot...
0,1820099.docx,Unsupported File / Scanned PDF / Password Prot...
0,2390009.pdf,Unsupported File / Scanned PDF / Password Prot...
0,2811113.pdf,Unsupported File / Scanned PDF / Password Prot...
0,5060070.pdf,Unsupported File / Scanned PDF / Password Prot...
0,8101049.pdf,Unsupported File / Scanned PDF / Password Prot...
0,Bill.Partner_2018-05-17-161628.pdf,Unsupported File / Scanned PDF / Password Prot...


In [10]:
df1.to_csv(r'C:\ML Training Docs\test\test\files\result_searchable_files.csv',index=False)

In [11]:
df2.to_csv(r'C:\ML Training Docs\test\test\files\result_non-searchable_files.csv',index=False)

In [12]:
df3 = pd.concat([df1,df2],axis=0)

In [13]:
df3

,filename,Prediction_cat1,Prediction_cat2,score,Type
0,1710093.docx,3rd Parties,Trust / NHSLA,99.85,NaN
0,180014.pdf,Registration Body,GMC,72.15,NaN
0,1630070.docx,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...
0,1820099.docx,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...
0,2390009.pdf,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...
0,2811113.pdf,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...
0,5060070.pdf,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...
0,8101049.pdf,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...
0,Bill.Partner_2018-05-17-161628.pdf,NaN,NaN,NaN,Unsupported File / Scanned PDF / Password Prot...


In [91]:

for filename in os.listdir(path):
    file_path = os.path.join(path,filename)
    main_Function(file_path)

.docx
unsupported file
.docx
Searchable docx
.pdf
Searchable pdf
.docx
Scanned docx
.pdf
Scanned pdf
.pdf
unsupported file
.pdf
Scanned pdf
.pdf
unsupported file
.pdf
Password protected file


In [89]:
path= r'C:/ML Training Docs/test/test/files/'

In [90]:
filename(path)

C:/ML Training Docs/test/test/files/1630070.docx
C:/ML Training Docs/test/test/files/1710093.docx
C:/ML Training Docs/test/test/files/180014.pdf
C:/ML Training Docs/test/test/files/1820099.docx
C:/ML Training Docs/test/test/files/2390009.pdf
C:/ML Training Docs/test/test/files/2811113.pdf
C:/ML Training Docs/test/test/files/5060070.pdf
C:/ML Training Docs/test/test/files/8101049.pdf
C:/ML Training Docs/test/test/files/Bill.Partner_2018-05-17-161628.pdf


In [125]:
for file in os.listdir(path):
    print(file)
    
    if file.endswith(".pdf"):
        pdf=pdf_txt(path+file)

    if file.endswith(".PDF"):
        pdf1=pdf_txt(path+file)
    
    if file.endswith(".docx"):
        docx=docx_to_txt(path+file)
    
    if file.endswith(".pptx"):
        pptx=pptx_txt(path+file)
        
#     if file.endswith(".xls"):
#         xls=xls_txt(path)
        
#     if file.endswith(".xlsx"):
#         xlsx=xlsx_txt(path)
          

1630070.docx
unsupported file
1710093.docx
Searchable docx
180014.pdf
Searchable pdf
1820099.docx
Scanned docx
2390009.pdf
Scanned pdf
2811113.pdf
unsupported file
5060070.pdf
Scanned pdf
8101049.pdf
unsupported file
Bill.Partner_2018-05-17-161628.pdf
Password protected file


In [9]:
def main_Function(filepath):
    content = ''
#     wordcnt=0
    filename, file_extension = os.path.splitext(filepath)
    print(file_extension)
    if file_extension == '.docx':
        content = docx_to_txt(filepath)
    
    if file_extension.lower() == ".pdf":
        content=pdf_txt(filepath)
    
#     wordcnt = len(content.split())
    print(1)
  
    content = data_cleansing(content)
    count_vectorizer_saved=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\count_vectorizer_cv.sav','rb'))    
    test_data_features=count_vectorizer_saved.transform([content])
    tfidf_saved=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\term_freq_inv_df.sav','rb'))
    test_data_features_tfidf=tfidf_saved.transform(test_data_features)
    lgbm_pickled_model=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\light_gbm.sav','rb'))
    x=lgbm_pickled_model.predict(test_data_features_tfidf)
    se=pd.Series(x)
    category = {0:'3rd Parties',1:'GMC',2:'MDU_Contacts', 3:'MDU_invoice', 4:'NHSLA'}
    result=list(se.apply(lambda x: category[x]))
    score=lgbm_pickled_model.predict_proba(test_data_features_tfidf)
    cat2=result[0]
    cat1=""

    if cat2=='3rd Parties':
        cat1="Experts"
        cat2=='3rd Party Expert'
    if cat2=='GMC':
        cat1="Registration Body"
    if cat2=='MDU_Contacts':
        cat1="MDU Solicitor"
        cat2="Contacts"
    if cat2=='MDU_invoice':
        cat1="MDU Solicitor"
        cat2="Invoices"
    if cat2=='NHSLA':
        cat1="3rd Parties"        
        cat2="Trust / NHSLA"
        print(2)
    score=format(max(score[0]*100), '.2f')
    bss="No"
    if float(score)<=80:
        bss="Yes"
    df = pd.DataFrame({'filename':filename.split('/')[-1]+file_extension,'Prediction_cat1':cat1,'Prediction_cat2':cat2,'score':score}, index=[0])
    print(3)
    return df

In [44]:
df1

,filename,Prediction_cat1,Prediction_cat2,score
0,1710093.docx,3rd Parties,Trust / NHSLA,99.85
0,180014.pdf,Registration Body,GMC,72.15


In [27]:
for index, row in x.iterrows():
    print(row['filename'])

Bill.Partner_2018-05-17-161628.pdf


In [31]:
for i in range(x.shape[0]):
    print(i)

0


In [30]:
for i, row in enumerate(x.values):
    print(x)

                             filename  \
0  Bill.Partner_2018-05-17-161628.pdf   

                                                Type  
0  Unsupported File / Scanned PDF / Password Prot...  


In [10]:
x.to_csv(r'C:\ML Training Docs\test\test\result.csv')

In [79]:
root= r'C:/ML Training Docs/test/test/files/'
file=[]
for filename in os.listdir(root):
    path = os.path.join(root,filename)
    for i in path:
        file.append(i)
        print(file)
        
#     if os.path.isfile(path):
#         print(path)

['C']
['C', ':']
['C', ':', '/']
['C', ':', '/', 'M']
['C', ':', '/', 'M', 'L']
['C', ':', '/', 'M', 'L', ' ']
['C', ':', '/', 'M', 'L', ' ', 'T']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g', ' ']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g', ' ', 'D']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g', ' ', 'D', 'o']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g', ' ', 'D', 'o', 'c']
['C', ':', '/', 'M', 'L', ' ', 'T', 'r', 'a', 'i', 'n', 'i', 'n', 'g', ' ', 'D', 'o', 'c', 's']
['C', ':', '/', 'M', 'L', ' ', 

In [78]:
main_Function(path)

.pdf
Password protected file


,filename,Prediction_cat1,Prediction_cat2,score
0,Bill.Partner_2018-05-17-161628.pdf,Experts,3rd Parties,74.83


In [63]:
filename.split('/')[-1]+file_extension, cat1, cat2, score, bss

AttributeError: 'function' object has no attribute 'split'

In [67]:
path = r'C:/ML Training Docs/test/test/files/1630070.docx'

In [6]:
file_path= r'C:/ML Training Docs/test/test/files/'

In [7]:
for file_name in os.listdir(file_path):
    f = os.path.join(file_path,file_name)
    if os.path.isfile(f):
        #print(main_Func(f))
        print(f)

C:/ML Training Docs/test/test/files/1630070.docx
C:/ML Training Docs/test/test/files/1710093.docx
C:/ML Training Docs/test/test/files/180014.pdf
C:/ML Training Docs/test/test/files/1820099.docx
C:/ML Training Docs/test/test/files/2390009.pdf
C:/ML Training Docs/test/test/files/2811113.pdf
C:/ML Training Docs/test/test/files/5060070.pdf
C:/ML Training Docs/test/test/files/8101049.pdf
C:/ML Training Docs/test/test/files/Bill.Partner_2018-05-17-161628.pdf


In [14]:
def main_Function(filepath):
    content = ''
    wordcnt=0
    filename, file_extension = os.path.splitext(filepath)
    print(file_extension)
    if file_extension == '.docx':
        content = docx_to_txt(filepath)
    
    if file_extension.lower() == ".pdf":
        content=pdf_txt(filepath)
    
    wordcnt = len(content.split())
  
    content = data_cleansing(content)
    count_vectorizer_saved=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\count_vectorizer_cv.sav','rb'))    
    test_data_features=count_vectorizer_saved.transform([content])
    tfidf_saved=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\term_freq_inv_df.sav','rb'))
    test_data_features_tfidf=tfidf_saved.transform(test_data_features)
    lgbm_pickled_model=joblib.load(open(r'C:\ML Training Docs\New folder\pickle file\light_gbm.sav','rb'))
    x=lgbm_pickled_model.predict(test_data_features_tfidf)
    se=pd.Series(x)
    category = {0:'3rd Parties',1:'GMC',2:'MDU_Contacts', 3:'MDU_invoice', 4:'NHSLA'}
    result=list(se.apply(lambda x: category[x]))
    score=lgbm_pickled_model.predict_proba(test_data_features_tfidf)
    cat2=result[0]
    cat1=""

    if cat2=='3rd Parties':
        cat1="Experts"
        cat2=='3rd Party Expert'
    if cat2=='GMC':
        cat1="Registration Body"
    if cat2=='MDU_Contacts':
        cat1="MDU Solicitor"
        cat2="Contacts"
    if cat2=='MDU_invoice':
        cat1="MDU Solicitor"
        cat2="Invoices"
    if cat2=='NHSLA':
        cat1="3rd Parties"        
        cat2="Trust / NHSLA"
    score=format(max(score[0]*100), '.2f')
    bss="No"
    if float(score)<=80:
        bss="Yes"
    return cat1, cat2, score, file_extension,  bss

In [15]:
path = r'C:/ML Training Docs/test/test/files/1630070.docx'

In [16]:
main_Function(path)

.docx
unsupported file


AttributeError: 'NoneType' object has no attribute 'split'